In [8]:
import pandas as pd

# Read the Excel file into a DataFrame while performing the following operations:
# 1. Convert the 'd' column to numeric, setting non-convertible values to NaN.
# 2. Drop rows with NaN values in the 'd' column, as they can't be used for numerical comparisons.
# 3. Sort the DataFrame by the weight per unit length 'W' column in descending order.
df = (pd.read_excel('aisc-shapes-database-v16.0.xlsx', sheet_name='Database v16.0')
      .assign(d=lambda df_: pd.to_numeric(df_['d'], errors='coerce'))  # Convert 'd' column to numeric
      .dropna(subset=['d'])  # Remove rows with NaN in 'd' column
      .sort_values('W', ascending=False))  # Sort by 'W' in descending order

# Display the first few rows of the DataFrame to verify its structure and the success of the operations above.
print("First few rows of the DataFrame:", df.head(), sep='\n')

# Group the DataFrame by 'Type' and calculate the mean of 'W' (weight per unit length) for each type.
# This gives us the average weight per foot for each beam type.
avg_weight_per_type = df.groupby('Type')['W'].mean()
print("\nAverage weight per foot (W) for each beam type:", avg_weight_per_type, sep='\n')

# Use the DataFrame's query method to filter out rows where the depth 'd' is greater than 20 inches,
# and reset the index to account for the rows that have been filtered out.
deep_beams = df.query('d > 20').reset_index(drop=True)
print("\nBeams with a depth greater than 20 inches:", deep_beams.head(), sep='\n')

# Sum the 'W' column of the DataFrame to get the total weight of all beams.
# This assumes that 'W' represents the weight per unit length.
total_weight = df['W'].sum()
print(f"\nTotal weight of all beams: {total_weight} lbs/ft")


First few rows of the DataFrame:
    Type EDI_Std_Nomenclature AISC_Manual_Label T_F      W      A     d  \
30     W              W36X925           W36X925   T  925.0  272.0  43.1   
181    W              W14X873           W14X873   T  873.0  257.0  23.6   
31     W              W36X853           W36X853   T  853.0  251.0  43.1   
182    W              W14X808           W14X808   T  808.0  238.0  22.8   
32     W              W36X802           W36X802   T  802.0  236.0  42.6   

       ddet Ht  h  ... rts.1 ho.1  PA.1 PA2.1  PB.1  PC.1  PD.1  T.1 WGi.1  \
30   43.125  –  –  ...   135  980  3400     –  3890  2670  3150  822   191   
181  23.625  –  –  ...   153  460  2420     –  2900  1680  2150  286   216   
31   43.125  –  –  ...   133  980  3430     –  3890  2640  3100  822   191   
182   22.75  –  –  ...   151  450  2370     –  2840  1630  2100  286   216   
32   42.625  –  –  ...   131  973  3380     –  3840  2620  3070  822   191   

    WGo.1  
30   76.2  
181  76.2  
31   76.2  

In [10]:
# Assuming that 'bf' is the width of the flange, 'tf' is the thickness of the flange,
# 'd' is the overall depth, 'tw' is the thickness of the web, and the areas are not provided
# If the database provides the flange and web areas, you should use those instead of calculating them

# Convert relevant columns to numeric
columns_to_convert = ['d', 'bf', 'tf', 'tw', 'Ix', 'Iy']
df[columns_to_convert] = df[columns_to_convert].apply(pd.to_numeric, errors='coerce')

# Drop rows where any of the relevant columns could not be converted to a numeric type
df.dropna(subset=columns_to_convert, inplace=True)

# Calculate the plastic section modulus for x-axis (Zx) for a simplified I-beam section
df['Zx'] = 2 * (df['bf'] * df['tf']) * (df['d'] / 2 - df['tf'] / 2) + (df['d'] - 2 * df['tf']) * df['tw'] * df['tw'] / 2

# Display the DataFrame with the Moment of Inertia and Plastic Section Modulus
print("DataFrame with Moment of Inertia (Ix, Iy) and Plastic Section Modulus (Zx):")
print(df[['Type', 'Ix', 'Iy', 'Zx']].head())

# show the first few rows of the DataFrame to verify the calculations
print("\nFirst few rows of the DataFrame with Moment of Inertia and Plastic Section Modulus:", df.head(), sep='\n')

DataFrame with Moment of Inertia (Ix, Iy) and Plastic Section Modulus (Zx):
    Type       Ix      Iy           Zx
30     W  73000.0  4940.0  3405.060268
181    W  18100.0  6170.0  1971.550364
31     W  70000.0  4600.0  3288.026028
182    W  15900.0  5550.0  1771.543888
32     W  64800.0  4210.0  3054.649644

First few rows of the DataFrame with Moment of Inertia and Plastic Section Modulus:
    Type EDI_Std_Nomenclature AISC_Manual_Label T_F      W      A     d  \
30     W              W36X925           W36X925   T  925.0  272.0  43.1   
181    W              W14X873           W14X873   T  873.0  257.0  23.6   
31     W              W36X853           W36X853   T  853.0  251.0  43.1   
182    W              W14X808           W14X808   T  808.0  238.0  22.8   
32     W              W36X802           W36X802   T  802.0  236.0  42.6   

       ddet Ht  h  ... rts.1  ho.1  PA.1 PA2.1  PB.1  PC.1  PD.1  T.1 WGi.1  \
30   43.125  –  –  ...   135   980  3400     –  3890  2670  3150  822   191

In [11]:
import pandas as pd


# Convert all relevant columns to numeric types just in case they aren't already
columns_to_convert = ['bf', 'tf', 'd', 'tw', 'Zx']
df[columns_to_convert] = df[columns_to_convert].apply(pd.to_numeric, errors='coerce')

# Calculate the area of one flange (we assume two flanges)
df['A_f'] = df['bf'] * df['tf']

# Calculate the area of the web
df['A_w'] = (df['d'] - 2 * df['tf']) * df['tw']

# Assuming the PNA is at the centroid of the web and flanges yield in tension and compression equally
df['y_f'] = (df['d'] - df['tf']) / 2
df['y_w'] = df['tw'] / 2

# Calculate an approximate Zx (plastic section modulus about x-axis)
df['Zx_calc'] = 2 * df['A_f'] * df['y_f'] + df['A_w'] * df['y_w']

# Compare the calculated Zx to the provided Zx
df['Zx_diff'] = df['Zx_calc'] - df['Zx']

# Display the original Zx, the calculated Zx, and their difference
print(df[['Zx', 'Zx_calc', 'Zx_diff']].head())

              Zx      Zx_calc  Zx_diff
30   3405.060268  3405.060268      0.0
181  1971.550364  1971.550364      0.0
31   3288.026028  3288.026028      0.0
182  1771.543888  1771.543888      0.0
32   3054.649644  3054.649644      0.0
